# DEL 2 API fra yr.no

In [ ]:
# Importer bibliotekene vi trenger
import requests
import json
import pandas as pd
from datetime import datetime

# For automatiske tester trenger vi pytest og ipytest
import ipytest
ipytest.autoconfig()

## Oppgave 3: Introduksjon til Yr.no API

Nå skal vi begynne å arbeide med ekte værdata! Yr.no tilbyr gratis værdata gjennom sitt API.

**API dokumentasjon:** https://api.met.no/weatherapi/locationforecast/2.0/documentation
Gå inn på lenken over og ha denne tilgjengelig mens du jobber videre. Siden inneholder API-spesifiksjonen, og forteller utviklere hvordan de kan bruke yr sin locationforecast api. 

**Base-adresse:** REST APIer bruker å ha en base-adresse, som kommer først for alle uri-er knyttet til APIet. 
For locations apiet til yr er baseadressen `https://api.met.no/weatherapi/locationforecast/2.0/`. Ofte, som her,
inkluderer base-uri et versjonsnummer. Slik kan APIet oppdateres samtidig som den gamle versjonen er tilgjengelig,
slik at utviklere har tid til å oppdatere sine programmer til å støtte spesifikasjon i ny versjon av APIet.

> Yr har flere APIer som du kan utforske på https://api.met.no
> Vi skal bruke loactionforecast som eksempel her

La oss prøve et enkelt kall til APIet, med uri `https://api.met.no/weatherapi/locationforecast/2.0/status`. 
Dette skal gi oss en status-beskjed, og kan brukes for å sjekke om vi klarer å koble oss opp mot APIet.

In [ ]:
response_3a = requests.get('https://api.met.no/weatherapi/locationforecast/2.0/status')
print("Status kode:", response_3a.status_code)

Oops.. Her fikk vi en feil statuskode - 403

---

### Oppgave 3a
Hva betyr denne statuskoden?

_Skriv svaret ditt her_

---

Hvis vi ser i dokumentasjonen, står det i delen 
[AUTHENTICATION](https://api.met.no/weatherapi/locationforecast/2.0/documentation#AUTHENTICATION) 
at en er nødt til å bruke en unik identifikator i User-Agent header-verdi for at ikke forespørselen
skal avvises.

La oss sjekke hvordan våre header-verdier så ut fra requesten vi sendte over

> Tips:
> Informasjon om sendte forespørsler kan hentes ut fra respons-variabelen
> med egenskapen `request` (eks: `respons_variabel.request`)

In [ ]:
# Her skriver vi ut headere som ble sendt med forespørselen
# Kjør cellen og se på output
print("\nRequest headers:")
request_headers = response_3a.request.headers
for key, value in request_headers.items():
    print(f"{key}: {value}")

---

### Oppgave 3b

Hva er verdien til headeren _User-Agent_?

_Skriv svaret ditt her_

---

Verdien på _User-Agent_ oppfyller ikke vilkårene til i API-spesifikasjonen. La oss prøve igjen der vi setter en unik verdi for User-Agent:

In [ ]:
# Kjør cellen under for å gjøre et nytt kall med en gyldig User-Agent header

# Sett en unik User-Agent header som beskrevet i dokumentasjonen
headers = {
    'User-Agent': 'TestApp/1.0 (it-test@sanpro.no)'
}

# Gjør forespørselen med de nye headerne
response = requests.get('https://api.met.no/weatherapi/locationforecast/2.0/status', headers=headers)

print("Status kode:", response.status_code)

# Skriv ut headere som ble sendt med forespørselen
print("\nRequest headers:")
print(json.dumps(dict(response.request.headers), indent=2))

# Hvis statuskoden er 200 (ok), skriv ut JSON-responsen
if response.status_code == 200:
    data = response.json()
    print("\nRespons data:")
    print(json.dumps(data, indent=2))
else:
    print("Noe gikk galt med forespørselen.")

Nå fikk vi et gyldig svar! 

---

### Oppgave 3c

Hva inneholdt responsen av data?

_Skriv svaret ditt her_

---

> Tips ISO 8601
> Les om standarden ISO 8601 for dato og tid formatert som tekst på [Wikipedia](https://en.wikipedia.org/wiki/ISO_8601)

Som vi nå har erfart er det lurt å sjekke dokumentasjon på API før bruk, og også vilkår. Les yr sine [Terms of Service her](https://api.met.no/doc/TermsOfService)

**Viktige punkter om Yr.no API:**
- Du må identifisere applikasjonen din med en "User-Agent" header
- Du kan ikke sende for mange forespørsler (maks 20 per sekund)
- API-et krever geografiske koordinater (bredde- og lengdegrad)

## Oppgave 4: Et enkelt kall til locationforecast

Siden det er noen verdier vi kommer til å bruke flere ganger videre, kan det være lurt å lagre disse til noen variabler først. 

Kjør cellen under for at variablene som defineres skal være tilgjengelig senere.

In [ ]:
# Sett opp korrekte headers for Yr.no API
headers = {
    'User-Agent': 'TestApp/1.0 (it-test@sanpro.no)'  # Bytt gjerne ut med din e-post
}

# URL for Yr.no API
base_url = "https://api.met.no/weatherapi/locationforecast/2.0/"
base_url_compact = base_url + "compact"
base_url_complete = base_url + "complete"

# Utskrift av base-verdiene
print("API URL:", base_url)
print("API url for compact metoden:", base_url_compact)
print("API url for complete metoden:", base_url_complete)
print("Headers:", headers)

_Kjør cellen under for å teste om variablene er satt opp riktig_

In [ ]:
%%ipytest
# Test at variablene er satt opp riktig
def test_api_setup():
    assert base_url == "https://api.met.no/weatherapi/locationforecast/2.0/"
    assert base_url_compact == "https://api.met.no/weatherapi/locationforecast/2.0/compact"
    assert base_url_complete == "https://api.met.no/weatherapi/locationforecast/2.0/complete"
    assert 'User-Agent' in headers
    assert headers['User-Agent'] != 'python-requests/'  # Sjekk at User-Agent er unik

Da kan vi gjøre vårt første ordentlige kall til APIet:

In [ ]:

# Koordinater for Hana, Sandnes
lat = 58.854  # Breddegrad (N-S)
long = 5.767  # Lengdegrad (Ø-V)

# Bygg URL for API-forespørsel
url = f"{base_url_compact}?lat={lat}&lon={long}"

# Gjennomfør forespørselen
response = requests.get(url, headers=headers)
print("Status kode:", response.status_code)
if response.status_code == 200:
    print("Respons data (formatert):")
    data = response.json()
    print(json.dumps(data, indent=2)[:500], "...")  # Print de første 500 tegnene av responsen
else:
    print("Noe gikk galt med forespørselen.")

---

### Oppgave 4a - Lag din første værdata-forespørsel

Velg en lokasjon og hent værdata for denne lokasjonen med et enkelt API-kall. Skriv koden din under. 

**Lagre responsen i en variabel med navn `response_4a`**

_Tips: Bruk google maps til å finne koordinatene_

In [ ]:
# Skriv din kode her
# OBS! Lagre responsen i en variabel med navn response_4a, slik at vi kan bruke den i neste oppgave.

# Koordinater
lat = _
long = _

# Bygg URL for API-forespørsel
url = _

# Gjennomfør forespørselen
response_4a = _

_Kjør cellen under for å sjekke at du gjorde oppgaven over riktig_

In [ ]:
%%ipytest

def test_response_4a():
    assert 'response_4a' in globals(), "Variabelen response_4a finnes ikke."
    assert response_4a.status_code == 200, "Responsen i variabelen response_4a har ikke statuskode 200."


***

## Oppgave 5: Utforsk værdata-strukturen

Før vi kan bruke dataene, må vi forstå hvordan de er organisert.

In [ ]:
# Konverter til JSON og lagre til fil
if response_4a.status_code == 200:
    data_task4 = response_4a.json()
    with open('weather_data.json', 'w') as f:
        json.dump(data_task4, f, indent=2)
    print("Data lagret til weather_data.json")
else:
    print("Feil! Respons ugyldig.")
    print("Sjekk om du har gjennomført forrige oppgave korrekt, og kjørt cellen.")
    print("OG at du har lagret responsen i en variabel med navn response_4a.")

---

### Oppgave 5a

Åpne filen `weather_data.json` og se gjennom verdiene. Filen er ganske stor, men du vil se at strukturen gjentas etter 50-100 linjer, så du trenger bare å forstå starten for å skjønne hele innholdet. 

- Hvilke hovedegenskaper (1. nivå) er det i json-filen? (det skal være tre stk, navngi dem)
- Hvilke egenskaper finnes under "properties" ?
- Hvor finner du de faktiske værdataene i filen?

_Skriv svaret ditt her_

***

### Oppgave 5b - Se nærmere på strukturen til en enkelt prognose

Fullfør funksjonene `get_properties` og `get_timeseries` under. Husk å returnere en verdi som siste linje av funksjonen:

```python
def get_properties(data_locationforecast=data_task4):
    # ... din logikk
    return variabel_navn # Returner en verdi til slutt
```


In [ ]:
def get_properties(data_locationforecast=data_task4):
    """Hent ut properties fra responsen fra locationforecast compact API-et"""
    pass # Erstatt denne linjen med din kode

def get_timeseries(data_locationforecast=data_task4):
    """Hent ut prognoser fra responsen fra locationforecast compact API-et."""
    pass # Erstatt denne linjen med din kode


Kjør cellen over etter du har fullført funksjonene. Du vil se en grønn hake ✅, men det kommer ingen output. Det er ok, og forventet. 

Etter du har kjørt cellen over (som laster funksjonsreferansene inn i python-interpreterens minne), kan du bruke testen under for å sjekke om løsningen din er gyldig.

_Kjør cellen under for å sjekke løsningen din_

In [ ]:
%%ipytest 

def test_get_properties():
    properties = get_properties()
    assert isinstance(properties, dict), "get_properties skal returnere en dictionary."
    assert 'timeseries' in properties, "properties skal inneholde nøkkelen 'timeseries'."
    assert isinstance(properties['timeseries'], list), "properties['timeseries'] skal være en liste."
    
def test_get_timeseries():
    timeseries = get_timeseries()
    assert isinstance(timeseries, list), "get_timeseries skal returnere en liste."
    assert len(timeseries) > 0, "get_timeseries skal returnere en ikke-tom liste."
    first_entry = timeseries[0]
    assert 'time' in first_entry, "Hver entry i timeseries skal inneholde nøkkelen 'time'."
    assert 'data' in first_entry, "Hver entry i timeseries skal inneholde nøkkelen 'data'."


***

### Oppgave 5c: Utforsk prognosens datastruktur

Kjør cellen under og se på utskriften

In [ ]:
# Utforsk en enkelt prognose i detalj
def print_timeseries_keys():
    """Skriv ut alle nøklene i en enkelt prognose i timeseries-listen"""
    prognoser = get_timeseries()
    first_item = prognoser[0]
    def print_keys(d, prefix=''):
        for key, value in d.items():
            print(f"{prefix}{key}")
            if isinstance(value, dict):
                print_keys(value, prefix + '  ')
    print_keys(first_item)
    
print_timeseries_keys()

__Forklar utskriften over__

- Hvilken informasjon finnes i et timeseries objekt?

_Skriv ditt svar her_